In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline
import pickle
from tqdm import tqdm_notebook as tqdm

In [2]:
from emb.load_embeddings import DataEmbeddings

Using TensorFlow backend.


In [3]:
ls

2019 NLU Naluhodo Example.ipynb  emb/                   README.md
best_model.hdf5                  eval/                  ROC_Curve_Util.py
Confusion_Matrix_Util.py         main.py                slurm-911387.out
data/                            models/
Elaine_test.ipynb                Predict_Class_Util.py


## load data

Load upsample indices, upsamples the orginal datasets from `1.7k` to `30k` training data by bootstrapping and random split as `training`, `validation`, `test` using k-fold validation  methodology (3-fold) in this case.

In [4]:
data_index = pickle.load(open('data/data_upsample_index.p', 'rb'))

for key in data_index:
    print(key)

train
val
test


Load original full dataset `beeap_1.csv`.

In [5]:
beeap_1 = pd.read_csv('data/beeapfinal/beeap_1.csv')
beeap_1.drop('Unnamed: 0', axis=1, inplace=True)
for i in beeap_1.columns[1:]:
    beeap_1[i] = beeap_1[i].astype(np.float64, errors='ignore')
    
print(beeap_1.shape)
# beeap_1.head()

(1702, 26)


In [6]:
dataset1 = DataEmbeddings(data_index, nth_sample= 0)
X_train, y_train, X_valid, y_valid, X_test, y_test, embedding_layer = dataset1.load_data_embeddings(beeap_1, '/scratch/cc5048/DataSource/glove.6B.300d.txt')

846it [00:00, 8459.08it/s]

Creating data sample 1 out of 3...
max_len : 200
max_feature : 100000
embed_size : 300
Loading word vectors...


400000it [00:28, 13821.14it/s]


Found 400000 word vectors.
Loading email contents...
max sequence length: 36540
min sequence length: 58
median sequence length: 303
Found 26969 unique tokens.
Shape of data tensor: (34040, 200)
Filling pre-trained embeddings...
Finished.
Generated X_train, y_train, X_valid, y_valid, X_test, y_test, embedding_layer as class attributes.


In [7]:
from models.BiLSTM import BiLstm

In [8]:
bilstm_1 =  BiLstm()
bilstm_1.arguments['epochs'] = 2

Building BiLstm Models ...
{'max_len': 200, 'batch_size': 128, 'epochs': 100, 'learning_rate': 0.001, 'learning_rate_decay': 0, 'units': 128, 'drop_out_rate': 0.2, 'checkpoint_path': 'best_model.hdf5', 'early_stop_patience': 10}


In [9]:
bilstm_1.fit(X_train, y_train, X_valid, y_valid, embedding_layer)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 34040 samples, validate on 227 samples
Epoch 1/2
34040/34040 [==============================] - 20s 590us/step - loss: 0.2137 - acc: 0.9149 - val_loss: 0.2813 - val_acc: 0.8943

 ROC-AUC - epoch: 1 - score: 0.892094

Epoch 00001: val_loss improved from inf to 0.28129, saving model to best_model.hdf5
Epoch 2/2
34040/34040 [==============================] - 18s 523us/step - loss: 0.0418 - acc: 0.9881 - val_loss: 0.3893 - val_acc: 0.8965

 ROC-AUC - epoch: 2 - score: 0.883934

Epoch 00002: val_loss did not improve from 0.28129
Finished Building BiLstm Model as class attribute class.model


haven't done for Lstm model .. before going through the predictons

In [13]:
pred = bilstm_1.predict(X_test)

340/340 [==============================] - 0s 250us/step


In [11]:
# lstm_1 =  Lstm(dataset1.X_train, dataset1.y_train, dataset1.X_valid, dataset1.y_valid, dataset1.embedding_layer)
# lstm_1.arguments['epochs'] = 2

In [12]:
# lstm_1.build_model()

In [14]:
from eval.evaluation import pred_classes_dt, pred_classes_f1, plot_roc_curve, plot_confusion_matrix

In [15]:
train_pred = bilstm_1.predict(X_train)
y_pred_dt = pred_classes_dt(train_pred, y_train, np.average(predictions, axis=0))

34040/34040 [==============================] - 7s 199us/step


NameError: name 'predictions' is not defined